In [2]:
import pandas as pd

In [6]:
# Get addresses
get_denns = pd.read_csv(f"C:\\Users\\mjjyo\\OneDrive\\Desktop\\omdena\\dennsbiomarkt.csv")

In [7]:
# Preprocess data
filtered_df = get_denns[get_denns['STORE_NAME'] == 'Denns BioMarkt']

In [33]:
# Turn into list
addresses = list(filtered_df['STORE_ADDRESS'])

In [121]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

import undetected_chromedriver as uc 
import cfscrape
import time

In [122]:
# Make scraper that bypasses cloudflare
scraper = cfscrape.create_scraper()

# Create undetected chrome instance
options = uc.ChromeOptions()
options.headless = False
driver = uc.Chrome(options=options)

url = 'https://www.biomarkt.de/berlin-karl-marx-str-66/angebote'
driver.get(url)

time.sleep(4.1562)
# Click accept cookies
cookies_button = driver.find_element(By.XPATH, "//button[@id='CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll']")
ActionChains(driver).move_to_element(cookies_button).click().perform()


C:\Users\mjjyo\AppData\Local\Temp\ipykernel_12800\2867378036.py:6: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = False


In [123]:
def scrape_the_page(address):

    time.sleep(2.53426)
    # Select button to get market prompt
    market_prompt = driver.find_element(By.XPATH, "//button[contains(@class, 'absolute') and contains(@class, 'top-14') and contains(@class, 'w-12') and contains(@class, 'h-12') and contains(@class, 'no-button-styles')]")
    ActionChains(driver).move_to_element(market_prompt).click().perform()


    time.sleep(2.6731)

    # Select the input box
    market_input = driver.find_element(By.XPATH, "//input[@placeholder= 'PLZ, Ort oder Straße']")
    time.sleep(1.42)
    ActionChains(driver).click(market_input).key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL).send_keys(Keys.DELETE).perform()
    time.sleep(1.127834)

    ActionChains(driver).move_to_element(market_input).click().send_keys(address).perform()

    try:
        # Search for that address
        market_confirm = driver.find_element(By.XPATH, "//button[contains(@class, 'absolute') and contains(@class, 'bottom-0') and contains(@class, 'right-0') and contains(@class, 'h-14') and contains(@class, 'rounded-r-full') and contains(@class, 'text-lg') and contains(@class, 'pt-1') and contains(@class, 'pl-2') and contains(@class, 'pr-4') and contains(@class, 'cursor-pointer') and contains(@class, 'bg-lightGreen') and contains(@class, 'text-white') and contains(@class, 'hover:bg-green') and contains(@class, 'no-button-styles')]")
        ActionChains(driver).move_to_element(market_confirm).click().perform()


        time.sleep(6.5568)
        
            
        # Select best result
        store_select_button = driver.find_element(By.XPATH, "//button[contains(@class, 'market-search-select-button')]")
        ActionChains(driver).move_to_element(store_select_button).click().perform()

        time.sleep(3.6423)
    except NoSuchElementException:
        driver.get(url)
        return None
    

    # This keeps scrolling down and pressing more products until there is no option to anymore
    going = True
    while going is True:
        try:

            # Displaying all food items

            wait = WebDriverWait(driver, 10)
            element = wait.until(EC.visibility_of_element_located((By.XPATH, "//button[contains(text(), 'Mehr Angebote laden')]")))
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            time.sleep(3.235)
            driver.execute_script("window.scrollBy(0, -325)")



            time.sleep(2.2643)

            # click on the element
            element.click()
            time.sleep(4.5612)


        except:
            going =  False

            
    # select all food items
    all_food_items = driver.find_elements(By.XPATH, "//article[contains(@class, 'relative') and contains(@class, 'sm:w-1/2') and contains(@class, 'lg:w-1/3') and contains(@class, 'border-solid') and contains(@class, 'border-bgBrown') and contains(@class, 'bg-white') and contains(@class, 'w-full') and contains(@class, 'md:border-r-2') and contains(@class, 'border-b-2') and contains(@class, 'max-h-160') and contains(@class, 'flex') and contains(@class, 'flex-col') and contains(@class, 'justify-between')]")
    
    image_list = []
    name_list = []
    information_list = []
    price_list = []
    
    # Get image
    for item in all_food_items:
        image_element = item.find_elements(By.XPATH, ".//img")
        image_url = image_element[0].get_attribute('src')
        image_list.append(image_url)
    
    # Get text info and price
    food_information = item.find_elements(By.XPATH, "//div[@class ='w-full px-12']")
    food_price = item.find_elements(By.XPATH, "//div[@class ='w-full h-full']")
    
    # Process data
    for i in range(len(food_price)):
        food_item_info = food_information[i].text.split('\n')

        food_item_name = food_item_info[0]
        food_item_details = food_item_info[1]

        food_item_price = food_price[i].text.split('\n')[0]

        name_list.append(food_item_name)
        information_list.append(food_item_details)
        price_list.append(food_item_price)

    return (name_list, image_list, information_list, price_list)


In [124]:
final_data = {'TEAM_MEMBER': [], 'STORE_ADDRESS': [], 'PRODUCT_IMAGE': [], 'PRODUCT_NAME': [], 'PRODUCT_INFO': [], 'PRODUCT_PRICE': []}
final_df = pd.DataFrame(final_data)
final_df

,TEAM_MEMBER,STORE_ADDRESS,PRODUCT_IMAGE,PRODUCT_NAME,PRODUCT_INFO,PRODUCT_PRICE


In [125]:
# Final Program, for each address scrap the page and add to df
for address in addresses:
    items = scrape_the_page(address)
    if items is not None:
        names = items[0]
        images = items[1]
        information = items[2]
        prices = items[3]


        for i in range(len(names)):
            new_row = {'TEAM_MEMBER': 'Michael Johnson', 'STORE_ADDRESS': address, 'PRODUCT_IMAGE': images[i], 'PRODUCT_NAME': names[i], 'PRODUCT_INFO': information[i], 'PRODUCT_PRICE': prices[i]}

            final_df = final_df.append(new_row, ignore_index=True)
    else:
        continue

C:\Users\mjjyo\AppData\Local\Temp\ipykernel_12800\1145182751.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(new_row, ignore_index=True)
C:\Users\mjjyo\AppData\Local\Temp\ipykernel_12800\1145182751.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(new_row, ignore_index=True)
C:\Users\mjjyo\AppData\Local\Temp\ipykernel_12800\1145182751.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(new_row, ignore_index=True)
C:\Users\mjjyo\AppData\Local\Temp\ipykernel_12800\1145182751.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.app

In [126]:
final_df

,TEAM_MEMBER,STORE_ADDRESS,PRODUCT_IMAGE,PRODUCT_NAME,PRODUCT_INFO,PRODUCT_PRICE
0,Michael Johnson,Hanne-Sobek-Platz · In the Berlin-Gesundbrunne...,https://cdn.sanity.io/images/or2m2oma/producti...,Brokkoli,1 kg,"3,99"
1,Michael Johnson,Hanne-Sobek-Platz · In the Berlin-Gesundbrunne...,https://cdn.sanity.io/images/or2m2oma/producti...,Erdbeeren,"500-g-Schale(1 kg = 5,98)","2,99"
2,Michael Johnson,Hanne-Sobek-Platz · In the Berlin-Gesundbrunne...,https://cdn.sanity.io/images/or2m2oma/producti...,Grapefruit,je Stück,"1,29"
3,Michael Johnson,Hanne-Sobek-Platz · In the Berlin-Gesundbrunne...,https://cdn.sanity.io/images/or2m2oma/producti...,Lauchzwiebeln,Bund,"1,99"
4,Michael Johnson,Hanne-Sobek-Platz · In the Berlin-Gesundbrunne...,https://cdn.sanity.io/images/or2m2oma/producti...,Maracuja,"je 100 g (1 kg = 19,90)","1,99"
...,...,...,...,...,...,...
2862,Michael Johnson,Chausseestraße 21,https://images.dennree.de/fitinto/300x200/5564...,DENNREE,"Tomaten-Passata ""Rustica""","1,29"
2863,Michael Johnson,Chausseestraße 21,https://cdn.sanity.io/images/or2m2oma/producti...,LOVECHOCK,Vegane Rohkostschokolade,"2,79"
2864,Michael Johnson,Chausseestraße 21,https://cdn.sanity.io/images/or2m2oma/producti...,ALLOS,Veganer Brotaufstrich,"2,00"
2865,Michael Johnson,Chausseestraße 21,https://images.dennree.de/fitinto/300x200/6066...,LOTAO,Veggie Hack,"2,79"


In [128]:
final_df.to_csv(f"C:\\Users\\mjjyo\\OneDrive\\Desktop\\omdena\\dennsbiomarktproducts.csv")